In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X_train = pd.read_csv('/kaggle/input/gstn-hackathon/X_Train_Data_Input_Imputed.csv')
y_train = pd.read_csv('/kaggle/input/gstn-hackathon/Y_Train_Data_Target.csv')
X_test = pd.read_csv('/kaggle/input/gstn-hackathon/X_Test_Data_Input_Imputed.csv')
y_test = pd.read_csv('/kaggle/input/gstn-hackathon/Y_Test_Data_Target.csv')
y_train = y_train.drop('ID', axis=1)
y_test = y_test.drop('ID', axis=1)

In [ ]:
import optuna
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import gc
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, confusion_matrix, log_loss, balanced_accuracy_score)

from optuna.samplers import TPESampler
from optuna.visualization import (plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour,
                                  plot_param_importances, plot_intermediate_values)
import plotly.io as pio

In [ ]:
os.makedirs('/kaggle/working/hypertuned_model_plots', exist_ok=True)
os.makedirs('/kaggle/working/optuna_visualizations/BalancedRandomForest', exist_ok=True)
os.makedirs('/kaggle/working/optuna_visualizations/EasyEnsemble', exist_ok=True)
os.makedirs('/kaggle/working/optuna_visualizations/RUSBoost', exist_ok=True)
os.makedirs('/kaggle/working/optuna_visualizations/Bagging', exist_ok=True)
os.makedirs('/kaggle/working/hypertuned_models', exist_ok=True)
os.makedirs('/kaggle/working/optuna_logs', exist_ok=True)
os.makedirs('/kaggle/working/xgboost_models', exist_ok=True)

In [ ]:
hypertuned_models = {}

# Helper function to plot AUC-ROC curve with metrics
def plot_roc_curve(fpr, tpr, auc_roc, name, accuracy, precision, recall, f1):
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC-ROC (AUC = {auc_roc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title(f'AUC-ROC Curve - {name}\n'
              f'Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig(f'/kaggle/working/hypertuned_model_plots/{name}_roc_curve.png')
    plt.close()

# Helper function to plot confusion matrix with metrics
def plot_confusion_matrix(cm, name, accuracy, precision, recall, f1):
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {name}\n'
              f'Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(f'/kaggle/working/hypertuned_model_plots/{name}_confusion_matrix.png')
    plt.close()

# Helper function to save Optuna visualizations as interactive HTML files
def save_optuna_visualizations(study, model_name):
    viz_dir = f'/kaggle/working/optuna_visualizations/{model_name}'
    os.makedirs(viz_dir, exist_ok=True)
    
    # Save interactive visualizations
    opt_hist = plot_optimization_history(study)
    pio.write_html(opt_hist, f'{viz_dir}/{model_name}_optimization_history.html')
    
    parallel_coord = plot_parallel_coordinate(study)
    pio.write_html(parallel_coord, f'{viz_dir}/{model_name}_parallel_coordinate.html')
    
    slice_plot = plot_slice(study)
    pio.write_html(slice_plot, f'{viz_dir}/{model_name}_slice.html')
    
    contour_plot = plot_contour(study)
    pio.write_html(contour_plot, f'{viz_dir}/{model_name}_contour.html')
    
    param_importances = plot_param_importances(study)
    pio.write_html(param_importances, f'{viz_dir}/{model_name}_param_importances.html')

    # New: Intermediate values plot (visualizes objective value per step of trials)
    intermediate_values_plot = plot_intermediate_values(study)
    pio.write_html(intermediate_values_plot, f'{viz_dir}/{model_name}_intermediate_values.html')

# Optuna tuning and training
def tune_and_train_model(model_name, train_func, param_func):
    def objective(trial):
        params = param_func(trial)  # Define the search space
        clf = train_func(params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy  # Maximize accuracy in the objective function
    
    # Use TPESampler (efficient for hyperparameter optimization)
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    
    # Run extensive tuning with 100 trials and no time constraint
    print(f"Starting tuning for {model_name}")
    study.optimize(objective, n_trials=40, timeout=None)
    
    # Get the best trial parameters
    best_params = study.best_params
    best_accuracy = study.best_value
    best_model = train_func(best_params)
    
    # Retrain the best model with best hyperparameters
    best_model.fit(X_train, y_train)
    
    # Predict and calculate metrics
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    
    # Plot AUC-ROC and Confusion Matrix
    plot_roc_curve(fpr, tpr, auc_roc, model_name, accuracy, precision, recall, f1)
    plot_confusion_matrix(cm, model_name, accuracy, precision, recall, f1)
    
    # Save the best model using pickle
    model_path = f'/kaggle/working/hypertuned_models/{model_name}.pkl'
    with open(model_path, 'wb') as model_file:
        pickle.dump(best_model, model_file)
    
    # Store in dictionary
    hypertuned_models[model_name] = {
        'best_params': best_params,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm.tolist(),
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'thresholds': thresholds.tolist(),
        'model_pickle': model_path
    }
    
    # Log the study results to a file
    log_path = f'/kaggle/working/optuna_logs/{model_name}_optuna_log.txt'
    with open(log_path, 'w') as log_file:
        log_file.write(study.trials_dataframe().to_csv(index=False))
    import json
    with open('/kaggle/working/result.json', 'a') as fp:
        json.dump(hypertuned_models, fp)
    
    # Save Optuna visualizations
    save_optuna_visualizations(study, model_name)
    
    print(f"{model_name} tuning complete. Best accuracy: {best_accuracy:.10f}")

In [ ]:
import catboost as cb


# Function to train CatBoost
def train_catboost(params):
    return cb.CatBoostClassifier(**params, silent=True)


def catboost_params(trial):
    return {
        'iterations': trial.suggest_int('iterations', 100, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }


In [ ]:
tune_and_train_model('CatBoost', train_catboost, catboost_params)